# Tutorial 4

Here we build a circuit and do a DC analysis, with a sweep of input voltages.


### Set Up

Lets get some intital imports out the way, and set auto reload, incase we change any of the external utils etc.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys

%load_ext autoreload
%autoreload 2

Now, `pyspice` imports, and setting up a logger.

In [ ]:
import PySpice
import PySpice.Logging.Logging as Logging
from PySpice.Spice.Netlist import Circuit
from PySpice.Unit import *

logger = Logging.setup_logging()

Might need to change the simulator...

In [ ]:
# # # change sim program location depending on system
# if sys.platform == "linux" or sys.platform == "linux2":
#     PySpice.Spice.Simulation.CircuitSimulator.DEFAULT_SIMULATOR = 'ngspice-subprocess'  # needed for linux
# elif sys.platform == "win32":
#     # You will get logging errors/warning, but is should work
#     pass

## Part 1: Simple DC Sweep




Create the circuit

In [ ]:
circuit = Circuit('Tutorial 4.1')

# # Define the 1N4148PH (Signal Diode)
circuit.model('MyDiode', 'D', IS=4.352@u_nA, RS=0.6458@u_Ohm, BV=110@u_V, IBV=0.0001@u_V, N=1.906)

# # add components to the circuit
circuit.V('input', 1, circuit.gnd, 10@u_V)
circuit.Diode(1, 1, 2, model='MyDiode')
circuit.R(1, 2, circuit.gnd, 1@u_kOhm)  # @u_kΩ is a unit of kOhms

print(circuit)

We can now create our simulation.

In [ ]:
simulator = circuit.simulator(temperature=25, nominal_temperature=25)

Now, we run a DC voltage sweep. Here, apply a `slice()` as the sweep to the `input` node, used in the Voltage source component, which resolves to: `V<node>`.

In [ ]:
analysis = simulator.dc(Vinput=slice(0, 5, 0.1))

The sweep of input values can be found in `analysis.sweep`

In [ ]:
analysis.sweep

As we would expect, the input sweep is the same as the resulting voltages on node `1`.

In [ ]:
(analysis.sweep == analysis["1"]).all()

We can plot some of our results.

In [ ]:
plt.plot(analysis["1"], analysis["2"])
plt.xlabel("Input Voltage (node 1)")
plt.ylabel("Output Voltage (node 2)")

## Part 2: B-Source Components

Here, we some an example using a B voltage soure.

In this example, it is used to create a series of voltage selected voltage outputs.

First, lets set up our circuit. \
We use a voltage source to run a voltage sweep on an 'imaginary' node.
This is used to control the B-source.

In [ ]:
# # create the circuit
circuit2 = Circuit('Tutorial 4.2')

# # Define the 1N4148PH (Signal Diode)
circuit2.model('MyDiode', 'D', IS=4.352@u_nA, RS=0.6458@u_Ohm, BV=110@u_V, IBV=0.0001@u_V, N=1.906)

# # valtage source
circuit2.V('i', 'img', circuit2.gnd, 0@u_V)


circuit2.Diode(1, 1, 2, model='MyDiode')
circuit2.R(1, 2, circuit2.gnd, 1@u_kOhm)  # @u_kΩ is a unit of kOhms

Now, lets create a 'look up table' of sorts, to create the B-sourse output behaviour associated with the imaginaty voltage.

In [ ]:
data = [0,1,5,6,2,4,4.5,5]
v_seq = "pwl(v(img),"
loop = 1
for val in data:

    # If the line gets too big, then move to a new line safely
    if loop % 10 == 0:
        v_seq += os.linesep + '+'

    # Create pwl voltage mapping
    if loop == len(data):
        v_seq += ' %d,%.2f' % (loop, val)
    else:
        v_seq += ' %d,%.2f,' % (loop, val)
    loop += 1
v_seq += ')'

v_seq

In [ ]:
circuit2.B('custom_source', 1, circuit2.gnd, v=v_seq)

print(circuit2)

In [ ]:
simulator1 = circuit2.simulator(temperature=25, nominal_temperature=25)

In [ ]:
analysis2 = simulator1.dc(Vi=slice(1, len(data)-1, 1))
analysis2.sweep

In [ ]:
plt.plot(np.array(analysis2["img"]), np.array(analysis2["1"]))
plt.xlabel("Imaginay Node Voltage sweep (node img)")
plt.ylabel("B-Source Voltage (node 1)")
plt.title('B-Source Behaviour')

In [ ]:
plt.plot(np.array(analysis2["1"]), np.array(analysis2["2"]))
plt.xlabel("B-Source Voltage (node 1)")
plt.ylabel("Output Voltage (node 2)")
plt.title('Circuit Behaviour')

In [ ]:
v_seq

In [ ]:
from utils.methods import format_analysis

format_analysis(analysis2, cast=True)